In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests
from config import *
from copy import deepcopy
from datetime import datetime
import polars as pl
from pathlib import Path
import numpy as np




# Configs

In [3]:
DATA_FOLDER = Path('data')


############## REGION - Get Payment
REGION_PARAMS = {
    "sdk":"1.28.3"
}
REGIONS_API = 'https://on-ramp-content.api.cx.metamask.io/regions'
REGIONS = [
    {
        "region":"de"
        , "country":"DE"
        , "fiat_currency": "/currencies/fiat/eur"
    },
    {
        "region":"us-fl"
        , "country":"US"
        , "fiat_currency": "/currencies/fiat/usd"
    },
    {
        "region":"gb"
        , "country":"UK"
        , "fiat_currency": "/currencies/fiat/gbp"
    }
]

############## QUOTE
QUOTE_API = 'https://on-ramp.api.cx.metamask.io/providers/all/quote'
CRYPTOCURRENCIES = [
    {
        "name": "Ethereum Mainnet"
        , "id": "/currencies/crypto/1/0x0000000000000000000000000000000000000000"
    },
    {
        "name": "USDT (Ethereum)"
        , "id":  "/currencies/crypto/1/0xdac17f958d2ee523a2206206994597c13d831ec7"
    },
    {
        "name": "USDT (BNB Chain)"
        , "id":  "/currencies/crypto/56/0x55d398326f99059ff775485246999027b3197955"
    },
    
]


# Scrape Payment data for regions

We can get all the payments providers for a given region instead of manually looking them up. 
In an ongoing process this would only be needed to run when a new region is added or on-demand

In [4]:
def extract_payments_data(payment_dict:dict):
    payment_dict.pop('logo', None)
    payment_dict.pop('icons', None)
    return payment_dict

def add_region_payment_data(region_dict:str, region_api:str = REGIONS_API, query_params:dict = REGION_PARAMS):
    # using a deepcopy to avoid mutating the original input
    dict_copy = deepcopy(region_dict)
    response = requests.get(f"{region_api}/{dict_copy['region']}", params=query_params)
    try:
        data = response.json()
        payment_info = [extract_payments_data(x) for x in data["payments"]]
        dict_copy["payments"] = payment_info
        print("Add Payment data")
        return dict_copy
    except Exception as e:
        print("Error adding payment data")
        print(e)

In [5]:
regions_with_payment = []
for region in REGIONS:
    print(f"extracting payment info for", region)
    region_it = add_region_payment_data(region)
    regions_with_payment.append(region_it)    

extracting payment info for {'region': 'de', 'country': 'DE', 'fiat_currency': '/currencies/fiat/eur'}
Add Payment data
extracting payment info for {'region': 'us-fl', 'country': 'US', 'fiat_currency': '/currencies/fiat/usd'}
Add Payment data
extracting payment info for {'region': 'gb', 'country': 'UK', 'fiat_currency': '/currencies/fiat/gbp'}
Add Payment data


# Generate Quote Scrapings

Now, we generate a list of all the query parameters needed (except the amount) to make a call to the API. In an ongoing process this could be persisted to a table and run on demand (same considerations as before)

In [6]:
quote_base_params = []
for region in regions_with_payment:
    for payment in region["payments"]:
        for crypto in CRYPTOCURRENCIES:
            quote_base_params.append({
                "country": region["country"]
                , "region": region["region"] 
                , "crypto_name": crypto["name"]
                , "payment_name": payment["name"]
                , "scrape_params": {
                    "regionId":f"/regions/{region['region']}"
                    , "paymentMethodId": payment["id"]
                    , "cryptoCurrencyId": crypto["id"]
                    , "fiatCurrencyId": region["fiat_currency"]
                }
                
            })

# Scraping a quote

the following sections define the logic to finally make a call, parse the response and persist to a parquet file

In [25]:
def get_success_quote_data(provider_json) -> dict:
    # print(provider_json)
    quote_data = {}
    # quote_data["provider"] = provider_json.get("provider")
    
    fields_to_scrape = [ {"parents": [], "key":"provider", "default":None}
        , {"parents": ["quote"], "key":"amountIn", "default":None}
        , {"parents": ["quote"], "key":"amountOut", "default":None}
        , {"parents": ["quote"], "key":"exchangeRate", "default":None}
        , {"parents": ["quote"], "key":"networkFee", "default":None}
        , {"parents": ["quote"], "key":"providerFee", "default":None}
        , {"parents": ["quote"], "key":"extraFee", "default": None}
        , {"parents": ["quote"], "key":"bestRate", "default":False}
        , {"parents": ["quote", "crypto"] , "key":"decimals", "default":None}
        , {"parents": ["quote", "crypto"] , "key":"name", "default":None}
        , {"parents": ["quote", "crypto"] , "key":"symbol", "default":None}
        , {"parents": ["quote", "crypto", "network"], "key":"chainId", "default":None}
        , {"parents": ["quote", "crypto", "network"], "key":"chainName", "default":None}
        , {"parents": ["quote", "crypto", "network"], "key":"shortName", "default":None}
    ]
        
    # populate the field
    for field in fields_to_scrape:
        it_json = deepcopy(provider_json)
        if len(field["parents"]) > 0 :
            for parent in field["parents"]:
                it_json = it_json.get(parent, {})
        quote_data[field["key"]] = it_json.get(field["key"],field["default"])
        
    # quote_data["amountOutFiat"] = quote_data.get("amountOut",*quote_data["exchangeRate"]
    # quote_data["totalFee"] = quote_data["amountIn"] - quote_data["amountOutFiat"]
    # quote_data["spread"] = quote_data["totalFee"] - quote_data["networkFee"] - quote_data["providerFee"] - quote_data["extraFee"]
    return quote_data
            
                    
    
    # # this is to keep the dict filling with the same pattern
    # # for both error and success dict
    # json_ref = provider_json
    # if isinstance(provider_json.get("quote"), dict):
    #     json_ref = provider_json.get("quote")
    # for field in quote_fields_to_scrape:
    #     quote_data[field["key"]] = json_ref.get(field["key"], field["default"])
    # return quote_data

    
def _format_string(x):
    return x.replace("(","").replace(")", "").replace(" ", "-").lower()

def generate_filename(query_params:dict, amount, format="parquet"):
    keys_to_format = ["region", "crypto_name", "payment_name"]
    keys_formatted = [_format_string(query_params[x]) for x in keys_to_format]
    keys_formatted.append(str(amount))
    keys_formatted.append(query_params["scrape_params"]["fiatCurrencyId"].split("/")[-1])
    keys_formatted.insert(0, datetime.now().strftime("%Y%m%d_%H%M"))
    return f'{"__".join(keys_formatted)}.{format}'


def scrape_one_quote(query_params: dict , amount: float, endpoint=QUOTE_API) -> dict:
    # print(query_params)
    copy_params = deepcopy(query_params)
    scrape_params = copy_params.pop("scrape_params")
    scrape_params["amount"] = amount
    scrape_params["request_time"] = datetime.now()
    response = requests.get(endpoint, params=scrape_params)
    
    # add them at the same level to keep tabular format
    copy_params.update(scrape_params)
    print(response.status_code)
    # return response
    """ 
    I don't want to assume that a provider that is not in the "success" means it does not
    support the currency, it probably was just a error at this specific quoting
    Logging the error for completeness.
    It will also make coverage counting easier later on...
    """
    try:
        raw = response.json()
        all_quotes = []
        i=1
        for key in ["success", "error"]:
            if len(raw[key]) > 0:
                for provider_quote in raw[key]:
                    single_quote = get_success_quote_data(provider_quote)
                    single_quote["position"] = i if key == 'success' else None
                    single_quote["has_quote"] = True if key =='success' else False     
                    single_quote.update(copy_params)
                    all_quotes.append(single_quote)
                    if key == 'success':
                        i += 1
        return {"status": "success", "payload": all_quotes}
    except Exception as e: 
        print("Error scraping", copy_params)
        print(e)
        print()
        return {"status": "failure", "payload": response}

def scrape_and_persist_quote(quote_params:dict, amount:float, endpoint:str, data_folder:Path):
    
    quote_schema = {
        "provider":pl.String
        , "amountIn": pl.Float64
        , "amountOut": pl.Float64
        , "exchangeRate": pl.Float64
        , "networkFee": pl.Float64
        , "providerFee": pl.Float64
        , "extraFee": pl.Float64
        , "bestRate": pl.Boolean
        , "decimals": pl.Int32
        , "name": pl.String
        , "symbol": pl.String
        , "chainId": pl.String
        , "chainName": pl.String
        , "shortName": pl.String
        # , "amountOutFiat": pl.String
        # , "totalFee": pl.Float32
        # , "spread": pl.Float32
        , "position": pl.Int32
        , "has_quote": pl.Boolean
        , "regionId": pl.String 
        , "country": pl.String 
        , "region": pl.String
        , "crypto_name": pl.String
        , "payment_name": pl.String
        , "paymentMethodId":pl.String 
        , "cryptoCurrencyId":pl.String
        , "fiatCurrencyId": pl.String 
        , "amount":pl.Float64
        , "request_time": pl.Datetime
    } 
    
    
    scrape_result = scrape_one_quote(
        quote_params
        , amount=amount
        , endpoint=endpoint
    )
    
    if scrape_result["status"] == 'success':
        filename = generate_filename(quote_params, amount)
        (
            pl.DataFrame(scrape_result["payload"], schema = quote_schema)
            .with_columns(
                ( pl.col("amountOut") * pl.col("exchangeRate") ).alias("amountOutFiat")
                , (pl.col("amountIn") -  pl.col("amountOut") * pl.col("exchangeRate") ).alias("totalFee")
            )
            .with_columns(
                ( pl.col("totalFee") - pl.col("networkFee") - pl.col("providerFee") - pl.col("extraFee")).alias("spreadFee")
            )
            .write_parquet(data_folder / filename, compression="snappy")
        )

    return scrape_result


# Bulk scraping

At this point we have a full scraping process for a single price point, we now need to scrape all the points we want to

In [26]:
def calculate_bin_midpoints(min_value=30, max_value=30000, n_bins=20):
    bins = [min_value * (max_value / min_value)**(i/n_bins) for i in range(n_bins + 1)]
    bins = [round(b) for b in bins]

    midpoints = []
    for i in range(len(bins) - 1):
        from_value = bins[i]
        to_value = bins[i + 1]
        midpoint = round((from_value + to_value) / 2)
        midpoints.append({"from": from_value, "midpoint": midpoint, "to": to_value})

    return midpoints

# Example usage:
bin_midpoints = calculate_bin_midpoints()
bin_midpoints

[{'from': 30, 'midpoint': 36, 'to': 42},
 {'from': 42, 'midpoint': 51, 'to': 60},
 {'from': 60, 'midpoint': 72, 'to': 85},
 {'from': 85, 'midpoint': 102, 'to': 119},
 {'from': 119, 'midpoint': 144, 'to': 169},
 {'from': 169, 'midpoint': 204, 'to': 238},
 {'from': 238, 'midpoint': 288, 'to': 337},
 {'from': 337, 'midpoint': 406, 'to': 475},
 {'from': 475, 'midpoint': 574, 'to': 672},
 {'from': 672, 'midpoint': 810, 'to': 949},
 {'from': 949, 'midpoint': 1144, 'to': 1340},
 {'from': 1340, 'midpoint': 1616, 'to': 1893},
 {'from': 1893, 'midpoint': 2284, 'to': 2674},
 {'from': 2674, 'midpoint': 3226, 'to': 3777},
 {'from': 3777, 'midpoint': 4556, 'to': 5335},
 {'from': 5335, 'midpoint': 6436, 'to': 7536},
 {'from': 7536, 'midpoint': 9090, 'to': 10644},
 {'from': 10644, 'midpoint': 12840, 'to': 15036},
 {'from': 15036, 'midpoint': 18137, 'to': 21238},
 {'from': 21238, 'midpoint': 25619, 'to': 30000}]

In [12]:
import time 
from tqdm import tqdm

total_iterations = len(bin_midpoints) * len(quote_base_params)
with tqdm(total=total_iterations, desc="Scraping quotes") as pbar:
    for bin in bin_midpoints:
        for quote_param in quote_base_params:
            r = scrape_one_quote(
                quote_params=quote_param
                , amount=bin["midpoint"]
                , endpoint=QUOTE_API
                , data_folder=DATA_FOLDER
            )
            # to not overload the API
            time.sleep(0.2)
            pbar.update(1)

Scraping quotes:   0%|          | 0/1020 [00:00<?, ?it/s]

200


Scraping quotes:   0%|          | 1/1020 [00:02<41:47,  2.46s/it]

200


Scraping quotes:   0%|          | 2/1020 [00:05<45:02,  2.65s/it]

...

Scraping quotes: 100%|█████████▉| 1019/1020 [29:36<00:01,  1.51s/it]

500
Error scraping {'country': 'UK', 'region': 'gb', 'crypto_name': 'USDT (Ethereum)', 'payment_name': 'PSE', 'regionId': '/regions/gb', 'paymentMethodId': '/payments/pse', 'cryptoCurrencyId': '/currencies/crypto/1/0xdac17f958d2ee523a2206206994597c13d831ec7', 'fiatCurrencyId': '/currencies/fiat/gbp', 'amount': 25619, 'request_time': datetime.datetime(2024, 8, 2, 22, 12, 34, 685500)}
'success'



Scraping quotes: 100%|██████████| 1020/1020 [29:38<00:00,  1.88s/it]

500
Error scraping {'country': 'UK', 'region': 'gb', 'crypto_name': 'USDT (BNB Chain)', 'payment_name': 'PSE', 'regionId': '/regions/gb', 'paymentMethodId': '/payments/pse', 'cryptoCurrencyId': '/currencies/crypto/56/0x55d398326f99059ff775485246999027b3197955', 'fiatCurrencyId': '/currencies/fiat/gbp', 'amount': 25619, 'request_time': datetime.datetime(2024, 8, 2, 22, 12, 35, 816674)}
'success'



Scraping quotes: 100%|██████████| 1020/1020 [29:38<00:00,  1.74s/it]


In [29]:
quote_base_params[20]

{'country': 'US',
 'region': 'us-fl',
 'crypto_name': 'USDT (BNB Chain)',
 'payment_name': 'Debit or Credit',
 'scrape_params': {'regionId': '/regions/us-fl',
  'paymentMethodId': '/payments/debit-credit-card',
  'cryptoCurrencyId': '/currencies/crypto/56/0x55d398326f99059ff775485246999027b3197955',
  'fiatCurrencyId': '/currencies/fiat/usd'}}

In [22]:
r = scrape_and_persist_quote(quote_base_params[20]
                    , amount= 36                
                    , endpoint=QUOTE_API
                    , data_folder=DATA_FOLDER
                    )

200


# Load to BigQuery

In [24]:
(
    pl.read_parquet("data/*.parquet")
)

provider,amountIn,amountOut,exchangeRate,networkFee,providerFee,extraFee,bestRate,decimals,name,symbol,chainId,chainName,shortName,position,has_quote,regionId,country,region,crypto_name,payment_name,paymentMethodId,cryptoCurrencyId,fiatCurrencyId,amount,request_time,amountOutFiat,TotalFee,spreadFee
str,f64,f64,f64,f64,f64,f64,bool,i32,str,str,str,str,str,i32,bool,str,str,str,str,str,str,str,str,f64,datetime[μs],f64,f64,f64
"""/providers/unlimitmeld""",36.0,34.261049,0.99851,0.2,1.59,0.0,true,18,"""Tether USD""","""USDT""","""56""","""BNB Chain Mainnet""","""BNB Chain""",1,true,"""/regions/us-fl""","""US""","""us-fl""","""USDT (BNB Chain)""","""Debit or Credit""","""/payments/debit-credit-card""","""/currencies/crypto/56/0x55d398…","""/currencies/fiat/usd""",36.0,2024-08-03 10:54:09.817135,34.21,1.79,-8.8818e-16
"""/providers/moonpay""",36.0,34.0,1.019118,0.01,1.34,0.0,false,18,"""Tether USD""","""USDT""","""56""","""BNB Chain Mainnet""","""BNB Chain""",2,true,"""/regions/us-fl""","""US""","""us-fl""","""USDT (BNB Chain)""","""Debit or Credit""","""/payments/debit-credit-card""","""/currencies/crypto/56/0x55d398…","""/currencies/fiat/usd""",36.0,2024-08-03 10:54:09.817135,34.65,1.35,1.3323e-15
"""/providers/moonpay-b""",36.0,34.0,1.019118,0.01,1.34,0.0,false,18,"""Tether USD""","""USDT""","""56""","""BNB Chain Mainnet""","""BNB Chain""",3,true,"""/regions/us-fl""","""US""","""us-fl""","""USDT (BNB Chain)""","""Debit or Credit""","""/payments/debit-credit-card""","""/currencies/crypto/56/0x55d398…","""/currencies/fiat/usd""",36.0,2024-08-03 10:54:09.817135,34.65,1.35,1.3323e-15
"""/providers/transak-b""",36.0,33.98,1.01648,0.2,1.26,0.0,false,18,"""Tether USD""","""USDT""","""56""","""BNB Chain Mainnet""","""BNB Chain""",4,true,"""/regions/us-fl""","""US""","""us-fl""","""USDT (BNB Chain)""","""Debit or Credit""","""/payments/debit-credit-card""","""/currencies/crypto/56/0x55d398…","""/currencies/fiat/usd""",36.0,2024-08-03 10:54:09.817135,34.54,1.46,8.8818e-16
"""/providers/transak""",36.0,32.8,1.017073,0.2,2.44,0.0,false,18,"""Tether USD""","""USDT""","""56""","""BNB Chain Mainnet""","""BNB Chain""",5,true,"""/regions/us-fl""","""US""","""us-fl""","""USDT (BNB Chain)""","""Debit or Credit""","""/payments/debit-credit-card""","""/currencies/crypto/56/0x55d398…","""/currencies/fiat/usd""",36.0,2024-08-03 10:54:09.817135,33.36,2.64,4.4409e-16
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""/providers/koywe""",null,null,null,null,null,null,false,null,null,null,null,null,null,null,false,"""/regions/us-fl""","""US""","""us-fl""","""USDT (BNB Chain)""","""Debit or Credit""","""/payments/debit-credit-card""","""/currencies/crypto/56/0x55d398…","""/currencies/fiat/usd""",36.0,2024-08-03 10:54:09.817135,null,null,null
"""/providers/robinhood""",null,null,null,null,null,null,false,null,null,null,null,null,null,null,false,"""/regions/us-fl""","""US""","""us-fl""","""USDT (BNB Chain)""","""Debit or Credit""","""/payments/debit-credit-card""","""/currencies/crypto/56/0x55d398…","""/currencies/fiat/usd""",36.0,2024-08-03 10:54:09.817135,null,null,null
"""/providers/stripe""",null,null,null,null,null,null,false,null,null,null,null,null,null,null,false,"""/regions/us-fl""","""US""","""us-fl""","""USDT (BNB Chain)""","""Debit or Credit""","""/payments/debit-credit-card""","""/currencies/crypto/56/0x55d398…","""/currencies/fiat/usd""",36.0,2024-08-03 10:54:09.817135,null,null,null
